In [ ]:
import os
import cv2
import dlib
import pickle
import random
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from datetime import datetime
from imutils import face_utils

In [ ]:
base_dir = "."
checkpoint_path = os.path.join(base_dir, '../logs/model/siamese-1')

In [ ]:
'''
The following code cell is taken from the source code of keras_vggface.'
I tried using the preprocess_input function provided by tf.keras but they provide different results.
To my knowledge, it seems that the mean values which are subtracted in each image are different.
'''
K = tf.keras.backend

def preprocess_input(x, data_format=None, version=1):
    x_temp = np.copy(x)
    if data_format is None:
        data_format = K.image_data_format()
    assert data_format in {'channels_last', 'channels_first'}

    if version == 1:
        if data_format == 'channels_first':
            x_temp = x_temp[:, ::-1, ...]
            x_temp[:, 0, :, :] -= 93.5940
            x_temp[:, 1, :, :] -= 104.7624
            x_temp[:, 2, :, :] -= 129.1863
        else:
            x_temp = x_temp[..., ::-1]
            x_temp[..., 0] -= 93.5940
            x_temp[..., 1] -= 104.7624
            x_temp[..., 2] -= 129.1863

    elif version == 2:
        if data_format == 'channels_first':
            x_temp = x_temp[:, ::-1, ...]
            x_temp[:, 0, :, :] -= 91.4953
            x_temp[:, 1, :, :] -= 103.8827
            x_temp[:, 2, :, :] -= 131.0912
        else:
            x_temp = x_temp[..., ::-1]
            x_temp[..., 0] -= 91.4953
            x_temp[..., 1] -= 103.8827
            x_temp[..., 2] -= 131.0912
    else:
        raise NotImplementedError

    return x_temp

In [ ]:
vggface = tf.keras.models.Sequential()
vggface.add(tf.keras.layers.Convolution2D(64, (3, 3), activation='relu', padding="SAME", input_shape=(224,224, 3)))
vggface.add(tf.keras.layers.Convolution2D(64, (3, 3), activation='relu', padding="SAME"))
vggface.add(tf.keras.layers.MaxPooling2D((2,2), strides=(2,2)))
 
vggface.add(tf.keras.layers.Convolution2D(128, (3, 3), activation='relu', padding="SAME"))
vggface.add(tf.keras.layers.Convolution2D(128, (3, 3), activation='relu', padding="SAME"))
vggface.add(tf.keras.layers.MaxPooling2D((2,2), strides=(2,2)))
 
vggface.add(tf.keras.layers.Convolution2D(256, (3, 3), activation='relu', padding="SAME"))
vggface.add(tf.keras.layers.Convolution2D(256, (3, 3), activation='relu', padding="SAME"))
vggface.add(tf.keras.layers.Convolution2D(256, (3, 3), activation='relu', padding="SAME"))
vggface.add(tf.keras.layers.MaxPooling2D((2,2), strides=(2,2)))
 
vggface.add(tf.keras.layers.Convolution2D(512, (3, 3), activation='relu', padding="SAME"))
vggface.add(tf.keras.layers.Convolution2D(512, (3, 3), activation='relu', padding="SAME"))
vggface.add(tf.keras.layers.Convolution2D(512, (3, 3), activation='relu', padding="SAME"))
vggface.add(tf.keras.layers.MaxPooling2D((2,2), strides=(2,2)))
 
vggface.add(tf.keras.layers.Convolution2D(512, (3, 3), activation='relu', padding="SAME"))
vggface.add(tf.keras.layers.Convolution2D(512, (3, 3), activation='relu', padding="SAME"))
vggface.add(tf.keras.layers.Convolution2D(512, (3, 3), activation='relu', padding="SAME"))
vggface.add(tf.keras.layers.MaxPooling2D((2,2), strides=(2,2)))

vggface.add(tf.keras.layers.Flatten())

vggface.add(tf.keras.layers.Dense(4096, activation='relu'))
vggface.add(tf.keras.layers.Dropout(0.5))
vggface.add(tf.keras.layers.Dense(4096, activation='relu'))
vggface.add(tf.keras.layers.Dropout(0.5))
vggface.add(tf.keras.layers.Dense(2622, activation='softmax'))

vggface.pop()
vggface.add(tf.keras.layers.Dense(128, use_bias=False))

for layer in vggface.layers[:-2]:
    layer.trainable = False

In [ ]:
class SiameseNetwork(tf.keras.Model):
    def __init__(self, vgg_face):
        super(SiameseNetwork, self).__init__()
        self.vgg_face = vgg_face
        
    @tf.function
    def call(self, inputs):
        image_1, image_2, image_3 =  inputs
        with tf.name_scope("Anchor") as scope:
            feature_1 = self.vgg_face(image_1)
            feature_1 = tf.math.l2_normalize(feature_1, axis=-1)
        with tf.name_scope("Positive") as scope:
            feature_2 = self.vgg_face(image_2)
            feature_2 = tf.math.l2_normalize(feature_2, axis=-1)
        with tf.name_scope("Negative") as scope:
            feature_3 = self.vgg_face(image_3)
            feature_3 = tf.math.l2_normalize(feature_3, axis=-1)
        return [feature_1, feature_2, feature_3]
    
    @tf.function
    def get_features(self, inputs):
        return tf.math.l2_normalize(self.vgg_face(inputs), axis=-1)

In [ ]:
model = SiameseNetwork(vggface)

In [ ]:
_ = model([tf.zeros((32,224,224,3)), tf.zeros((32,224,224,3)), tf.zeros((32,224,224,3))])
_ = model.get_features(tf.zeros((32,224,224,3)))

In [ ]:
checkpoint = tf.train.Checkpoint(model=model)
checkpoint.restore(checkpoint_path)

In [ ]:
data_dir = '../data/'

# Data Collection Details
The cell below should be re-run for data collection for multiple people.
For a single person, you could  collect 8-10 images tops. Entering the name of the person would add a directory to the data folder which will be the name of that person. 
If the name of the person exists it will give an error. So make sure to keep different names.

In [ ]:
name = input("Enter the name of the person : ")
os.mkdir(os.path.join(data_dir, name))
cap = cv2.VideoCapture(0)
count = 0
while True:
    ret, frame = cap.read()
    cv2.imshow('Image', frame)
    k = cv2.waitKey(1)
    if k == ord('s'):
        cv2.imwrite(os.path.join(data_dir, name + '/' + str(count) + '.png') , frame)
        count += 1
    if k ==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
features = []

In [ ]:
people = sorted(os.listdir(data_dir))
face_detector = dlib.get_frontal_face_detector()
features = []
dumpable_features = {}

In [ ]:
for person in people:
    person_path = os.path.join(data_dir, person)
    print(person_path)
    images = []
    for image in os.listdir(person_path):
        image_path = os.path.join(person_path, image)
        img = cv2.imread(image_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_detector(gray, 0)
        if len(faces) == 1:
            for face in faces:
                face_bounding_box = face_utils.rect_to_bb(face)
                if all(i >= 0 for i in face_bounding_box):
                    [x, y, w, h] = face_bounding_box
                    frame = img[y:y + h, x:x + w]
                    frame = cv2.resize(frame, (224, 224))
                    frame = np.asarray(frame, dtype=np.float64)
                    images.append(frame)
    images = np.asarray(images)
    images = preprocess_input(images)
    images = tf.convert_to_tensor(images)
    feature = model.get_features(images)
    feature = tf.reduce_mean(feature, axis=0)
    features.append(feature.numpy())
    dumpable_features[person] = feature.numpy()

# Dumping Features Rather than Calculating Them every time

I have added a variable dumpable_features which could be pickled. It will save a dictionary format of {'name':'features (numpy array)'} for each person. Simply pickle using the following commands
```
with open('weigths.pkl', 'wb') as f:
    pickle.dump(dumpable_features, f) 
```
and reload them using
```
with open('weigths.pkl', 'rb') as f:
    dumpable_features_reloaded = pickle.load(f)
    
people = []
features = []
for key, value in dumpable_features_reloaded.items():
    people.append(key)
    features.append(value)
```
During my testing I tested on 8-10 people so regenerating features wasn't that much slow. 

In [ ]:
features = np.asarray(features)

In [ ]:
print(people)

In [ ]:
cap = cv2.VideoCapture(0)
count = 0
name = 'not identified'
while True:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    faces = face_detector(gray, 0)
    for face in faces:
        face_bounding_box = face_utils.rect_to_bb(face)
        if all(i >= 0 for i in face_bounding_box):
            [x, y, w, h] = face_bounding_box
            frame = img[y:y + h, x:x + w]
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            frame = cv2.resize(frame, (224, 224))
            frame = np.asarray(frame, dtype=np.float64)
            frame = np.expand_dims(frame, axis=0)
            frame = preprocess_input(frame)
            feature = model.get_features(frame)
                
            dist = tf.norm(features - feature, axis=1)
            name = 'not identified'
            loc = tf.argmin(dist)
            if dist[loc] < 0.8:
                name = people[loc]
            else:
#                     print(dist.numpy())
                pass
                    
            font_face = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(img, name, (x, y-5), font_face, 0.8, (0,0,255), 3)
    cv2.imshow('Image', img)
    k = cv2.waitKey(1)
    if k ==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()